In [ ]:
# Installing the dependencies
!pip install fastapi
!pip install uvicorn
!pip install pydantic
!pip install pyngrok
!pip install nest-asyncio
!pip install --upgrade-strategy eager install optimum[onnxruntime]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 572.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 1.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 878.6 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.1/132.1 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pickle5: filename=pickle5-0.0.11-cp310-cp310-linux_x86_64.whl size=255314 sha256=df2fe378873fa829e002450eb1ffe8533a010261bcc1f6b992c0bd1f043ba740
  Stored in directory: /root/.cache/pip/wheels/7d/14/ef/4aab19d27fa8e58772be5c71c16add0426acf9e1f64353235c
Successfully built pickle5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.0/108.0 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.3/99.3 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.2/127.2 kB 10.1 MB/s eta 0:00:00


In [ ]:
# Import dependencies to run the API
from fastapi import FastAPI
from pydantic import BaseModel
from typing import List
import uvicorn
from starlette.middleware.cors import CORSMiddleware

# Import dependencies for the deployment
from pyngrok import ngrok, conf
import nest_asyncio

# Import dependencies for the model backend
from transformers import AutoTokenizer, pipeline
from optimum.onnxruntime import ORTModelForTokenClassification
from optimum.onnxruntime import ORTQuantizer
from optimum.onnxruntime.configuration import AutoQuantizationConfig

# Import dependencies for getting information about the Jupyter notebook
import os
from google.colab import userdata

In [ ]:
app = FastAPI()

In [ ]:
# This allows the frontend and backend on different servers to communicate with each other.
app.add_middleware(
    CORSMiddleware,
    allow_origins=["*"],
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)

In [ ]:
# Data validation using Pydantic
class NERRequest(BaseModel):
    text: str
class DataKeys(BaseModel):
    text: str
    ents: List
    title: None
class NERData(BaseModel):
    data_keys: DataKeys
class NERResponse(BaseModel):
    original: str
    name_entities: NERData

In [ ]:
# Load and save the model from the huggingface repo into the local runtime disk
tokenizerdis = AutoTokenizer.from_pretrained("elastic/distilbert-base-uncased-finetuned-conll03-english")
modeldis = ORTModelForTokenClassification.from_pretrained("elastic/distilbert-base-uncased-finetuned-conll03-english", export=True)

modeldis.save_pretrained('/save_directory')
tokenizerdis.save_pretrained('/save_directory')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/258 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Framework not specified. Using pt to export the model.


model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

Using the export variant default. Available variants are:
    - default: The default ONNX variant.

***** Exporting submodel 1/1: DistilBertForTokenClassification *****
Using framework PyTorch: 2.2.1+cu121
/usr/local/lib/python3.10/dist-packages/transformers/models/distilbert/modeling_distilbert.py:234: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  mask, torch.tensor(torch.finfo(scores.dtype).min)


('/save_directory/tokenizer_config.json',
 '/save_directory/special_tokens_map.json',
 '/save_directory/vocab.txt',
 '/save_directory/added_tokens.json',
 '/save_directory/tokenizer.json')

In [ ]:
# create ORTQuantizer and define quantization configuration
quantizer = ORTQuantizer.from_pretrained(modeldis)
quantizer_config = AutoQuantizationConfig.avx512_vnni(is_static=False, per_channel=False)

# apply the quantization configuration to the model and specify the directory to save the model
model_quantized_path = quantizer.quantize(
    save_dir="/model-quantized",
    quantization_config=quantizer_config,
)

Creating dynamic quantizer: QOperator (mode: IntegerOps, schema: u8/s8, channel-wise: False)
Quantizing model...
Saving quantized model at: /model-quantized (external data format: False)
Configuration saved in /model-quantized/ort_config.json


In [ ]:
class NEROnnxModel():
    '''
    Load the quantized distilbert model, run the pipeline, clean up the results and return the data
    '''
    def __call__(self, text: str) -> str:
        # load quantized model and tokenizers
        tokenizerdis = AutoTokenizer.from_pretrained("/save_directory")
        modeldis = ORTModelForTokenClassification.from_pretrained("/model-quantized")

        #create transformers pipeline
        onnx_ner = pipeline("token-classification", model=modeldis, tokenizer=tokenizerdis, aggregation_strategy='simple')

        #run the pipeline and return the results
        pred = onnx_ner(text)
        pred = self.sanitise_data(text, pred)
        return pred

    def sanitise_data(self, text: str, entities: list) -> dict:
        '''
        Remove extra key-value pairs from the result and return the cleaned dictionary
        :param text: original text sent by the user
        :param entities: list of entities returned by the model to be cleaned up
        '''
        return_entities=[]
        # Loop through every entity identofied by the model
        for entity in entities:
            santitise_entity={}
            santitise_entity['start'] = entity['start']
            santitise_entity['end'] = entity['end']
            santitise_entity['label'] = entity['entity_group']
            return_entities.append(santitise_entity) # Keep only the data needed by the front-end
        return {'text':text, 'ents':return_entities, 'title': None} # Required format

In [ ]:
# Loading the model class
ner_pipeline = NEROnnxModel()
print('The distilbert NER model & tokenizer sucessfully loaded!')

In [ ]:
# Routing for the GET request
@app.get("/")
def get_root():
    return "This is the RESTful API for PrivacyDetection"

The distilbert NER model & tokenizer sucessfully loaded!


In [ ]:
# Routing for the POST request
@app.post("/results", response_model=NERResponse)
async def predict(request: NERRequest):
    ner_text = NERData(data_keys=ner_pipeline(request.text)) # Sending the user's input to the model
    return NERResponse(
        original=request.text,
        name_entities=ner_text
    )

In [ ]:
# Authentication to run ngrok
os.environ["NGROK"] = userdata.get("NGROK")
conf.get_default().auth_token = os.environ["NGROK"]

In [ ]:
# Starting and running the ngrok and fastapi servers
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

INFO:     Started server process [2875]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


Public URL: https://c1d9-34-133-202-197.ngrok-free.app
INFO:     138.37.255.35:0 - "POST /results HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [2875]


KeyboardInterrupt: 

In [ ]:
# Killing the ngrok and fastapi servers
!pkill uvicorn
!pkill ngrok